In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms

from modules.UNet import *
from modules.Discriminator import *
from modules.DataSet import *
from modules.Losses import *

import os,sys

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from PIL import Image

import numpy as np

## Training

In [1]:
def train_adversarial(net,discriminator,device,val_per=0.1,epochs=10,batch_size=10,resize_to=None,alpha=1):
    if resize_to is not None:
        transform_image = transforms.Compose([
        transforms.Resize(resize_to),
        transforms.ToTensor(),
        transforms.Normalize(0.5,0.5)
        ])
        transform_label = transforms.Compose([
        transforms.Resize(resize_to),
        transforms.ToTensor()
        ])
    else:
        transform_image = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(0.5,0.5)
        ])
        transform_label = transforms.Compose([
        transforms.ToTensor()
        ])
            
    
    dataSet = UltraSoundDataSet(root_dir,(transform_image,transform_label))
    nTrain = int(len(dataSet)*(1-val_per))
    nValid = int(len(dataSet)-nTrain)
    
    trainSet,validSet = random_split(dataSet,[nTrain,nValid])
    
    train_loader = DataLoader(trainSet,batch_size=batch_size,shuffle=True,num_workers=4)
    valid_loader = DataLoader(validSet,batch_size=batch_size,shuffle=True,num_workers=4)
    
    optimizer_unet = torch.optim.Adam(net.parameters())
    optimizer_disc = torch.optim.Adam(discriminator.parameters())
    scheduler_unet = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_unet,mode='min',patience=10) #mae: dice-index
    scheduler_disc = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_disc,mode='min',patience=10)
    BCELoss = nn.BCELoss()
    
    running_loss_seg = 0
    running_loss_disc = 0
    step = 0
    np.set_printoptions(precision=2)
    
    isAdv = True;
    for epoch in range(epochs):
        net.train()
        discriminator.train()
        
        for batch in train_loader:
            imgs,labels = batch
            
            imgs = imgs.to(device=device,dtype=torch.float32)
            labels = labels.to(device=device,dtype=torch.float32)
            
            valid = torch.Tensor(imgs.size(0), 1).fill_(1.0).to(device=device,dtype=torch.float32)
            fake = torch.Tensor(imgs.size(0), 1).fill_(0.0).to(device=device,dtype=torch.float32)
            
            if(isAdv):
                # Train adversarial 
                with torch.no_grad():
                    # segmentation
                    pred = net(imgs)
                pred_valid = discriminator(imgs, labels)
                pred_fake = discriminator(imgs, pred.detach())
                print("valid: ",np.array(pred_valid.view([1,-1]).tolist()),"; fake: ",np.array(pred_fake.view([1,-1]).tolist()) )
                
                # discriminator loss
                disc_loss = BCELoss(pred_valid,valid) + BCELoss(pred_fake,fake)
                
                optimizer_disc.zero_grad()
                disc_loss.backward()
                optimizer_disc.step()
                
                running_loss_disc += disc_loss.item()
                
            else:
                # segmentation
                pred = net(imgs)
                with torch.no_grad():
                    pred_fake = discriminator(imgs, pred)
                print("fake: ",np.array(pred_fake.view([1,-1]).tolist()) )
                
                # segmentation loss
                seg_loss = BCELoss(pred,labels) + alpha*BCELoss(pred_fake,valid)
                
                optimizer_unet.zero_grad()
                seg_loss.backward()
                optimizer_unet.step()
                
                running_loss_seg += seg_loss.item()
              
            step += 1 
            if step % 10 == 9:    # print every 10 mini-batches
                if(isAdv):
                    print('[%d, %5d] discrimination loss: %.3f' %(epoch + 1, step + 1, running_loss_disc / 10))
                    running_loss_disc = 0.0
                else:
                    print('[%d, %5d] segmentation loss: %.3f' %(epoch + 1, step + 1, running_loss_seg / 10))  
                    running_loss_seg = 0.0
                    
            if step%100 == 99:
                val_loss_seg = 0
                val_loss_disc = 0
                for batch in valid_loader:
                    imgs,labels = batch
                    imgs = imgs.to(device=device,dtype=torch.float32)
                    labels = labels.to(device=device,dtype=torch.float32)
                    
                    with torch.no_grad():
                        pred_seg = net(imgs)
                        pred_valid = discriminator(imgs, labels)
                        pred_fake = discriminator(imgs, pred_seg.detach())
                    
                    val_loss_seg += (BCELoss(pred_seg,labels) + alpha*BCELoss(pred_fake,fake)).item()
                    valid = torch.Tensor(imgs.size(0), 1).fill_(1.0).to(device=device,dtype=torch.float32)
                    fake = torch.Tensor(imgs.size(0), 1).fill_(0.0).to(device=device,dtype=torch.float32)
                    val_loss_disc += (BCELoss(pred_valid,valid) + BCELoss(pred_fake,fake)).item()
                print('[%d, %5d] validation loss(seg): %.3f; validation loss(disc): %.3f' %(epoch + 1, step + 1, val_loss_seg / len(valid_loader), val_loss_disc / len(valid_loader)))
                scheduler_unet.step(val_loss_seg)
                scheduler_disc.step(val_loss_disc)
                
                if(isAdv):
                    scheduler_disc.step(val_loss_disc)
                else:
                    scheduler_unet.step(val_loss_seg)
                
                isAdv = not isAdv
                

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = UNet2(init_features=64)
discriminator = Discriminator()

net = net.to(device)
discriminator = discriminator.to(device)

IMG_SIZE = [256,256]
#IMG_SIZE = None

root_dir = os.path.expanduser("~/workspace/us_robot/DataSet/SimRealDataset")
try:
    #train(net=net,device=device,resize_to=None,epochs=50,batch_size=5)
    train_adversarial(net=net,discriminator=discriminator,device=device,resize_to=IMG_SIZE,epochs=20,batch_size=3)
except KeyboardInterrupt:
    sys.exit()

valid:  [[0.51 0.5  0.51]] ; fake:  [[0.5 0.5 0.5]]
valid:  [[0.78 0.76 0.78]] ; fake:  [[0.52 0.53 0.56]]
valid:  [[0.15 0.15 0.14]] ; fake:  [[0.02 0.02 0.02]]
valid:  [[0.95 0.97 0.97]] ; fake:  [[0.28 0.33 0.47]]
valid:  [[0.99 0.99 0.99]] ; fake:  [[0.56 0.43 0.42]]
valid:  [[0.99 0.99 0.99]] ; fake:  [[0.09 0.07 0.09]]
valid:  [[0.97 0.97 0.97]] ; fake:  [[0.01 0.01 0.01]]
valid:  [[0.95 0.96 0.96]] ; fake:  [[0.   0.02 0.  ]]
valid:  [[0.97 0.98 0.98]] ; fake:  [[0. 0. 0.]]
[1,    10] discrimination loss: 0.569
valid:  [[1.   0.99 0.99]] ; fake:  [[8.22e-05 3.27e-04 2.75e-04]]
valid:  [[0.99 1.   1.  ]] ; fake:  [[1.01e-03 7.00e-05 9.64e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[5.06e-05 6.18e-05 6.95e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[2.81e-05 3.08e-05 6.00e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[3.91e-04 1.67e-05 2.38e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[2.25e-04 7.24e-05 1.13e-04]]
valid:  [[1. 1. 1.]] ; fake:  [[1.72e-05 1.28e-05 6.39e-06]]
valid:  [[1. 1. 1.]] ; fake:  [[0. 0.

fake:  [[0.96 1.   1.  ]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
fake:  [[1.   0.97 1.  ]]
fake:  [[0.87 0.97 0.98]]
fake:  [[0.99 0.99 1.  ]]
[1,   160] segmentation loss: 0.242
fake:  [[0.46 1.   0.95]]
fake:  [[0.94 1.   1.  ]]
fake:  [[0.96 1.   1.  ]]
fake:  [[1.   0.91 0.89]]
fake:  [[1.   1.   0.97]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
fake:  [[0.99 1.   0.99]]
fake:  [[1.   1.   0.78]]
fake:  [[0.98 1.   1.  ]]
[1,   170] segmentation loss: 0.218
fake:  [[0.99 1.   1.  ]]
fake:  [[1.   0.96 1.  ]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
fake:  [[0.97 1.   1.  ]]
fake:  [[1.   0.99 1.  ]]
fake:  [[1.   1.   0.99]]
fake:  [[0.97 1.   1.  ]]
fake:  [[1. 1. 1.]]
[1,   180] segmentation loss: 0.158
fake:  [[1.   1.   0.98]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
fake:  [[0.99 1.   1.  ]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
fake:  [[1.   1.   0.99]]
fake:  [[1. 1. 1.]]
fake:  [[1. 1. 1.]]
[1,   190] segmentation lo

fake:  [[1.49e-17 1.44e-15 2.09e-15]]
fake:  [[9.65e-18 4.65e-12 1.89e-17]]
fake:  [[3.67e-16 1.10e-17 4.75e-18]]
fake:  [[7.39e-14 5.31e-18 2.18e-16]]
fake:  [[3.78e-16 1.11e-14 1.05e-17]]
fake:  [[2.59e-17 2.85e-15 1.26e-17]]
fake:  [[4.13e-17 1.13e-17 3.79e-16]]
fake:  [[8.32e-15 8.55e-19 1.09e-13]]
fake:  [[3.39e-12 5.36e-12 2.22e-11]]
[1,   320] segmentation loss: 35.036
fake:  [[1.71e-12 6.44e-14 1.39e-12]]
fake:  [[2.14e-14 4.64e-14 1.39e-13]]
fake:  [[1.16e-12 3.15e-18 1.56e-13]]
fake:  [[1.61e-17 5.31e-14 2.20e-18]]
fake:  [[1.27e-18 1.39e-17 1.93e-17]]
fake:  [[9.96e-15 1.30e-17 1.44e-17]]
fake:  [[6.44e-17 2.83e-17 8.43e-18]]
fake:  [[7.83e-18 7.37e-18 1.25e-16]]
fake:  [[9.01e-16 1.22e-17 3.61e-18]]
fake:  [[3.31e-16 7.66e-15 4.83e-17]]
[1,   330] segmentation loss: 35.497
fake:  [[1.62e-17 2.00e-18 2.52e-17]]
fake:  [[5.11e-16 4.91e-17 2.81e-17]]
fake:  [[2.76e-16 6.23e-13 9.63e-18]]
fake:  [[8.66e-13 6.66e-16 1.40e-17]]
fake:  [[1.16e-17 7.59e-18 6.51e-18]]
fake:  [[7.24e

valid:  [[1. 1. 1.]] ; fake:  [[1.06e-14 2.87e-11 5.57e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[1.72e-15 9.87e-18 7.26e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[1.16e-17 5.24e-11 1.38e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[2.00e-14 9.37e-17 3.48e-16]]
valid:  [[1. 1. 1.]] ; fake:  [[4.57e-17 7.11e-18 1.25e-16]]
valid:  [[1. 1. 1.]] ; fake:  [[7.08e-17 6.83e-12 2.94e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[9.05e-18 2.30e-16 5.35e-16]]
valid:  [[1. 1. 1.]] ; fake:  [[1.50e-17 1.24e-13 3.90e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[3.68e-13 2.11e-13 2.95e-17]]
valid:  [[1. 1. 1.]] ; fake:  [[1.43e-12 3.44e-13 3.68e-14]]
[1,   480] discrimination loss: 0.000
valid:  [[1. 1. 1.]] ; fake:  [[1.39e-17 1.60e-12 3.51e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[1.30e-12 3.02e-12 2.27e-18]]
valid:  [[1. 1. 1.]] ; fake:  [[6.88e-11 1.47e-11 2.84e-14]]
valid:  [[1. 1. 1.]] ; fake:  [[6.67e-10 9.96e-14 5.35e-15]]
valid:  [[1. 1. 1.]] ; fake:  [[3.08e-11 1.67e-10 4.09e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[6.33e-16 9.91e-

valid:  [[1. 1. 1.]] ; fake:  [[2.39e-06 8.05e-20 1.11e-06]]
valid:  [[1. 1. 1.]] ; fake:  [[3.67e-09 9.25e-05 1.46e-14]]
valid:  [[1. 1. 1.]] ; fake:  [[2.44e-14 3.73e-18 1.58e-19]]
valid:  [[1. 1. 1.]] ; fake:  [[6.20e-15 2.46e-08 5.01e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[5.44e-06 9.07e-07 2.39e-01]]
valid:  [[1. 1. 1.]] ; fake:  [[1.56e-07 7.59e-08 1.82e-21]]
valid:  [[0.8  0.98 1.  ]] ; fake:  [[3.43e-18 1.59e-20 1.84e-22]]
valid:  [[1. 1. 1.]] ; fake:  [[6.49e-12 5.76e-09 1.88e-11]]
valid:  [[1. 1. 1.]] ; fake:  [[1.62e-08 3.00e-20 1.80e-18]]
valid:  [[1. 1. 1.]] ; fake:  [[2.31e-05 3.56e-17 2.46e-18]]
[1,   640] discrimination loss: 0.017
valid:  [[1. 1. 1.]] ; fake:  [[2.00e-04 5.06e-20 1.74e-07]]
valid:  [[1. 1. 1.]] ; fake:  [[4.75e-11 4.45e-10 5.67e-15]]
valid:  [[1. 1. 1.]] ; fake:  [[5.10e-14 1.15e-06 3.29e-16]]
valid:  [[1. 1. 1.]] ; fake:  [[1.62e-04 2.54e-07 1.42e-04]]
valid:  [[1. 1. 1.]] ; fake:  [[2.49e-15 9.67e-05 6.21e-06]]
valid:  [[1. 1. 1.]] ; fake:  [[2.37e-17 

...

valid:  [[1. 1. 1.]] ; fake:  [[4.97e-06 2.86e-03 5.36e-06]]
valid:  [[1. 1. 1.]] ; fake:  [[1.04e-05 3.08e-34 1.25e-01]]
valid:  [[1. 1. 1.]] ; fake:  [[4.87e-05 1.48e-03 2.20e-06]]
valid:  [[1. 1. 1.]] ; fake:  [[2.72e-02 1.13e-04 5.38e-01]]
valid:  [[1.   1.   0.48]] ; fake:  [[1.23e-04 8.73e-01 3.05e-06]]
valid:  [[1.   0.99 0.98]] ; fake:  [[3.11e-01 7.88e-06 6.36e-04]]
valid:  [[1. 1. 1.]] ; fake:  [[3.24e-04 2.63e-01 8.52e-01]]
valid:  [[1. 1. 1.]] ; fake:  [[0.04 0.04 0.  ]]
valid:  [[1. 1. 1.]] ; fake:  [[1.13e-02 1.28e-08 2.61e-03]]
valid:  [[1. 1. 1.]] ; fake:  [[0.   0.17 0.01]]
[20, 17270] discrimination loss: 0.222
valid:  [[1. 1. 1.]] ; fake:  [[0.   0.01 0.  ]]
valid:  [[1. 1. 1.]] ; fake:  [[2.69e-07 7.50e-01 2.56e-03]]
valid:  [[1. 1. 1.]] ; fake:  [[1.11e-13 1.63e-02 9.94e-01]]
valid:  [[1. 1. 1.]] ; fake:  [[8.65e-06 7.19e-05 1.45e-03]]
valid:  [[1. 1. 1.]] ; fake:  [[0.05 0.   0.  ]]
valid:  [[0.73 1.   1.  ]] ; fake:  [[1.38e-09 8.69e-01 3.29e-01]]
valid:  [[1. 1.

valid:  [[1.   1.   0.25]] ; fake:  [[8.13e-05 6.19e-01 2.02e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[0.05 0.11 0.  ]]
[20, 17430] discrimination loss: 0.400
valid:  [[1. 1. 1.]] ; fake:  [[0. 0. 0.]]
valid:  [[0.99 1.   1.  ]] ; fake:  [[0.   0.   0.01]]
valid:  [[0.89 1.   1.  ]] ; fake:  [[1.09e-05 6.04e-03 1.72e-04]]
valid:  [[1. 1. 1.]] ; fake:  [[0.2 0.  0.3]]
valid:  [[1. 1. 1.]] ; fake:  [[0.   0.01 0.  ]]
valid:  [[1. 1. 1.]] ; fake:  [[4.31e-01 1.37e-07 9.98e-01]]
valid:  [[1.   1.   0.97]] ; fake:  [[9.95e-01 1.55e-01 6.09e-04]]
valid:  [[1. 1. 1.]] ; fake:  [[1.51e-04 6.89e-06 1.56e-03]]
valid:  [[1. 1. 1.]] ; fake:  [[0.65 0.06 0.01]]
valid:  [[0.34 1.   0.01]] ; fake:  [[6.67e-08 3.71e-03 7.25e-08]]
[20, 17440] discrimination loss: 0.691
valid:  [[1. 1. 1.]] ; fake:  [[1.28e-01 7.37e-06 2.25e-02]]
valid:  [[1. 1. 1.]] ; fake:  [[0.01 0.05 0.01]]
valid:  [[1. 1. 1.]] ; fake:  [[2.22e-02 2.96e-02 5.69e-16]]
valid:  [[0.98 1.   1.  ]] ; fake:  [[3.54e-04 1.53e-02 4.77e-01]]
val

fake:  [[0.   0.05 0.  ]]
fake:  [[0.13 0.07 0.01]]
fake:  [[2.16e-01 6.97e-06 6.18e-01]]
fake:  [[6.54e-04 1.64e-01 1.95e-05]]
fake:  [[3.70e-13 7.64e-06 1.38e-03]]
fake:  [[2.77e-01 1.48e-13 2.76e-01]]
fake:  [[6.05e-01 3.26e-05 6.52e-01]]
fake:  [[6.24e-03 1.72e-05 9.39e-05]]
fake:  [[6.69e-05 2.11e-03 3.53e-01]]
[20, 17600] segmentation loss: 7.314
[20, 17600] validation loss(seg): 0.425; validation loss(disc): 0.506
valid:  [[1. 1. 1.]] ; fake:  [[3.91e-01 2.48e-02 2.66e-13]]
valid:  [[1. 1. 1.]] ; fake:  [[8.19e-01 1.01e-02 8.86e-12]]
valid:  [[1. 1. 1.]] ; fake:  [[0.   0.75 0.02]]
valid:  [[0.98 1.   1.  ]] ; fake:  [[0.02 0.   0.  ]]
valid:  [[1. 1. 1.]] ; fake:  [[0.1 0.  0. ]]
valid:  [[1.   1.   0.99]] ; fake:  [[1.15e-12 8.99e-01 2.53e-07]]
valid:  [[0.25 1.   1.  ]] ; fake:  [[7.04e-07 2.63e-06 1.58e-04]]
valid:  [[1.   0.86 1.  ]] ; fake:  [[8.07e-06 2.93e-03 4.26e-03]]
valid:  [[1. 1. 1.]] ; fake:  [[0.76 0.02 0.44]]
valid:  [[1. 1. 1.]] ; fake:  [[3.27e-02 4.52e-02 5.2

fake:  [[9.17e-06 2.78e-01 6.07e-01]]
fake:  [[5.97e-03 4.68e-06 4.54e-01]]
fake:  [[6.28e-01 4.46e-08 4.00e-05]]
fake:  [[4.50e-01 1.30e-10 8.76e-01]]
fake:  [[8.86e-03 2.11e-03 1.21e-07]]
[20, 17740] segmentation loss: 6.363
fake:  [[1.15e-05 1.70e-05 1.17e-01]]
fake:  [[2.13e-03 8.46e-06 1.38e-02]]
fake:  [[0.14 0.   0.  ]]
fake:  [[1.82e-01 1.99e-07 9.88e-01]]
fake:  [[4.07e-05 2.89e-01 4.80e-02]]
fake:  [[7.01e-02 7.81e-06 3.19e-01]]
fake:  [[1.63e-02 6.88e-06 1.23e-07]]
fake:  [[3.06e-05 1.03e-02 9.61e-01]]
fake:  [[3.07e-02 9.58e-01 4.09e-08]]
fake:  [[9.59e-01 2.37e-04 5.39e-06]]
[20, 17750] segmentation loss: 6.575
fake:  [[7.00e-05 2.44e-01 9.72e-01]]
fake:  [[3.00e-07 3.87e-03 5.82e-06]]
fake:  [[6.47e-04 1.97e-22 2.61e-03]]
fake:  [[1.39e-08 1.43e-01 1.79e-02]]
fake:  [[2.94e-02 7.86e-01 3.01e-05]]
fake:  [[2.92e-06 1.23e-01 3.35e-02]]
fake:  [[1.04e-15 5.43e-04 9.53e-01]]
fake:  [[5.22e-06 3.93e-02 2.97e-03]]
fake:  [[3.23e-02 3.64e-05 1.13e-01]]
fake:  [[3.23e-24 1.55e-03

valid:  [[1. 1. 1.]] ; fake:  [[0. 0. 0.]]
valid:  [[1.   0.92 0.  ]] ; fake:  [[4.73e-02 1.47e-19 2.03e-06]]
[20, 17890] discrimination loss: 0.622
valid:  [[1. 1. 1.]] ; fake:  [[3.80e-04 6.20e-10 3.67e-02]]
valid:  [[1. 1. 1.]] ; fake:  [[2.76e-06 1.49e-02 6.64e-04]]
valid:  [[1. 1. 1.]] ; fake:  [[9.16e-01 1.20e-08 7.10e-05]]
valid:  [[1. 1. 1.]] ; fake:  [[0.95 0.65 0.48]]
valid:  [[1.   0.82 1.  ]] ; fake:  [[6.72e-03 4.55e-04 4.21e-05]]
valid:  [[1.   0.91 1.  ]] ; fake:  [[3.85e-04 2.55e-07 2.72e-04]]
valid:  [[1. 1. 1.]] ; fake:  [[0.09 0.92 0.39]]
valid:  [[1. 1. 1.]] ; fake:  [[0. 0. 0.]]
valid:  [[1. 1. 1.]] ; fake:  [[0.   0.39 0.14]]
valid:  [[1.   1.   0.99]] ; fake:  [[8.79e-01 1.47e-03 1.18e-04]]
[20, 17900] discrimination loss: 0.445
[20, 17900] validation loss(seg): 0.343; validation loss(disc): 0.425
fake:  [[5.62e-02 9.42e-08 4.10e-04]]
fake:  [[4.78e-06 9.21e-06 4.76e-07]]
fake:  [[7.76e-01 9.25e-09 4.41e-06]]
fake:  [[3.51e-04 2.12e-04 5.12e-01]]
fake:  [[0.   0.

In [6]:
#for python
torch.save(net.state_dict(), './unet_gan_usseg.pth')
torch.save(discriminator.state_dict(), './disc_gan_usseg.pth')
#for c++
traced_script_module = torch.jit.trace(net, img)
traced_script_module.save("./unet_gan_usseg_traced.pt")

## Inference

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PATH = os.path.expanduser("~/workspace/us_robot/unet_gan_usseg.pth")
net = UNet(init_features=32).to(device)
net.load_state_dict(torch.load(PATH))
net = net.eval()


In [9]:
test_dir = os.path.expanduser("~/workspace/us_robot/DataSet/realDataSet/linear/vessel")
pred_dir = os.path.expanduser("~/workspace/us_robot/DataSet/realDataSet/linear/vessel_pred")

testset_list = os.listdir(test_dir)
testset_list = list(filter(lambda x: x.endswith('jpg'), testset_list))
resize_to=[256,256]
transform_image = transforms.Compose([
    transforms.Resize(resize_to),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5) #Division by 255 is done, when the transformation assumes an image.
    ])
transform_label = transforms.Compose([
    transforms.Resize(resize_to),
    transforms.ToTensor()
    ])
invtransform_label = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([512,512])
    ])

for sample in testset_list:
    image_path = os.path.join(test_dir,sample)
    label_path = os.path.join(pred_dir,sample)
    
    img = Image.open(image_path).convert("L")
    img = transform_image(img)
    img = img.to(device)
    img = img.unsqueeze(0)
    #label = Image.open(label_path)
    #label = transform_label(label).to(device)
    #label = label.unsqueeze(0)
    
    
    with torch.no_grad():
        pred = net(img)
    
    #DiceIndex = (1 - DiceLoss(pred,label)).cpu().item()

    pred = invtransform_label(pred.cpu().squeeze(0))
    #fname = "pred%.2f.png"%DiceIndex
    fname = sample
    sav_path = os.path.join(pred_dir,fname)
    pred.save(sav_path)

In [8]:
test_dir = os.path.expanduser("~/workspace/us_robot/DataSet/SimRealDatasetTest")
testset_list = os.listdir(test_dir)
resize_to=[256,256]
transform_image = transforms.Compose([
    transforms.Resize(resize_to),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5) #Division by 255 is done, when the transformation assumes an image.
    ])
transform_label = transforms.Compose([
    transforms.Resize(resize_to),
    transforms.ToTensor()
    ])
invtransform_label = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([512,512])
    ])
for sample in testset_list:
    image_path = os.path.join(test_dir,sample,"image.png")
    label_path = os.path.join(test_dir,sample,"label.png")
    
    img = Image.open(image_path)
    img = transform_image(img)
    img = img.to(device)
    img = img.unsqueeze(0)
    label = Image.open(label_path)
    label = transform_label(label).to(device)
    label = label.unsqueeze(0)
    
    
    with torch.no_grad():
        pred = net(img)
    
    DiceIndex = (1 - DiceLoss(pred,label)).cpu().item()

    pred = invtransform_label(pred.cpu().squeeze(0))
    fname = "pred%.2f.png"%DiceIndex
    sav_path = os.path.join(test_dir,sample,fname)
    pred.save(sav_path)